In [1]:
from ase.build import bulk
from ase import Atoms
from ase.calculators.emt import EMT
from ase.constraints import FixAtoms
from ase.optimize import QuasiNewton
from ase.build import fcc111, add_adsorbate
from ase.spacegroup.symmetrize import *
from ase.spacegroup.xtal import crystal
from ase.spacegroup import Spacegroup
from ase.spacegroup import get_spacegroup
from ase.io import read, write
import ast
import glob
import numpy as np
from numpy import sqrt
from numpy import unique
from scipy.spatial.distance import pdist,cdist
import unittest


def read_Output(file):
    ifin = 0
    with open(file) as f:
        FINAL   = "FINAL OPTIMIZED GEOMETRY"
        INPUT   = "GEOMETRY FOR WAVE FUNCTION"
        SLAB    = " * TWO DIMENSIONAL SLAB PARALLEL TO THE SELECTED PLANE"
        SLABCUT = False

        
        if SLAB in f.read():
            SLABCUT = True
            print("TRUE")
            f.seek(0)
        
        if FINAL in f.read():
            f.seek(0)
            for i, line in enumerate(f.readlines()):
                if FINAL in line:
                    ifin = i 
        else:
            f.seek(0)
            for i, line in enumerate(f.readlines()):
                if INPUT in line:
                    ifin = i 
                           
        ilat = ifin +6
        inum = ifin +8
        ipos = ifin +11

        f.seek(0)

        data = f.readlines()
        
        a = float(data[ilat].split()[0])
        b = float(data[ilat].split()[1])
        c = float(data[ilat].split()[2])
        alpha = float(data[ilat].split()[3])
        beta  = float(data[ilat].split()[4])
        gamma = float(data[ilat].split()[5])

        atoms = int(data[inum].split()[-1])
        ecps  = []
        ele   = []
        fract = np.zeros((atoms,3)) 
        for j in range(ipos,ipos+atoms):
            k = j-ipos
            fract[k,0] = float(data[j].split()[-3])
            fract[k,1] = float(data[j].split()[-2])
            if SLABCUT == True:
                fract[k,2] = float(data[j].split()[-1])/500.
            else:
                fract[k,2] = float(data[j].split()[-1])
            ecps.append(int(data[j].split()[-5]))
            ele.append(data[j].split()[-4].title())  
    
    output = crystal(ele, fract, spacegroup =1, cell=[a,b,c,alpha,beta,gamma])        
    
    return(output)


    
class Crystal22():
    #Accepts an ASE-Python Crystal type object, material name and parameter file
    # NEED TO ACCOUNT FOR SPACEGROUPS WHEN WRITING UNIT CELL PARAMETERS
    ## - Also for d3 files
    ## - Also for Plotting Scripts
    ## 
    ## Add Getters and Setters/ property decorators 
    ##  - unitcell
    ##  - material
    ##  - ele
    ##  - fractional_coord
    ##  - crystal
    ######################################################################################
    def __init__(self,crystal,material='0',form='CRYSTAL',param='OPT'):
        self.crystal    = crystal
        self.material   = material
        self.sg         = get_spacegroup(self.crystal,symprec=1E-3).no
        self.form       = form
        if self.form == 'SLAB':
            self.sg = 1
        # '/Users/williamcomaskey/Desktop/CRYS22/opt.param'
        paramlib = "/Users/williamcomaskey/Desktop/Projects/crys22/library/parameters/"

        if param in ['OPT','opt','Opt']:
            fparam = paramlib+"opt.param"
            with open(fparam) as f:
                data = f.read()
            self.param = ast.literal_eval(data)
        
        elif param in ['1C','1c']:
            fparam = paramlib+"1c.param"
            with open(fparam) as f:
                data = f.read()
            self.param = ast.literal_eval(data)
            
        elif param in ['2C','2c']:
            fparam = paramlib+"2c.param"
            with open(fparam) as f:
                data = f.read()
            self.param = ast.literal_eval(data)
            
        elif param in ['SOC','soc','SC','sc']:
            fparam = paramlib+"soc.param"
            with open(fparam) as f:
                data = f.read()
            self.param = ast.literal_eval(data)
            
        else:
            print("Invalid Parameter Input File")
    
    def set_crystal(self,crystal):
        self.crystal    = crystal
        self.sg         = get_spacegroup(self.crystal,symprec=1E-3).no
        self.material   = self.crystal.get_chemical_formula()

        
    def read2Crystal(self,filename,fmt=""):
        if fmt =="":
            self.crystal = read(filename)
        else:
            self.crystal = read(filename,format=fmt)
        self.sg         = get_spacegroup(self.crystal,symprec=1E-3).no
  
    def Header(self): 
        if self.material == '0':
            #List out Elements and Make Label
            self.material = self.crystal.get_chemical_formula()
        
        #Make Header From Title
        if self.form == 'SLAB':
            d12_Header = "{}\nSLAB\n{}\n".format(self.material,self.sg)     
        else:
            d12_Header = "{}\nCRYSTAL\n0 0 0\n{}\n".format(self.material,self.sg)
        return(d12_Header)
    
    def Optimize(self):
        #THIS SHOULD BE DONE LAST AFTER EVERYTHING ELSE
        d12_Optimize = ""
        if self.param == '0':        
            d12_Optimize = "OPTGEOM\nFULLOPTG\nMAXCYCLE\n800\nENDOPT\nEND\n"     
        else:
            #Use Parameters from Input File 
            if self.param["SLABCUT"][0] == 1:
                d12_Optimize +="SLABCUT"+"\n"
                d12_Optimize += self.param["SLABCUT"][1]+"\n"
                d12_Optimize += self.param["SLABCUT"][2]+"\n"
                
            if self.param["Optimize"][0] == 1:
                for i in range(1,len(self.param["Optimize"])):
                    d12_Optimize +=self.param["Optimize"][i]+"\n"
            if self.param["MAXCYCLE"][0] ==1:
                d12_Optimize += "MAXCYCLE"+"\n"
                d12_Optimize += str(self.param["MAXCYCLE"][1])+"\n"                
                
            if self.param["MAXTRADIUS"][0] ==1:
                d12_Optimize += "MAXTRADIUS"+"\n"
                d12_Optimize += str(self.param["MAXTRADIUS"][1])+"\n"
            if (self.param["Symmetry"] ==0) or (self.param["TWOCOMP"] == 1) or (self.param["SOC"] == 1):
                d12_Optimize += "SYMMREMO"+"\n"
            d12_Optimize += "ENDOPT"+"\n"
        return(d12_Optimize)
      
    
    def Check_Sym(self):
        lattice = ""
        
        try:
            sym = get_spacegroup(self.crystal,symprec=1e-3).symbol[0]
            sg  = get_spacegroup(self.crystal,symprec=1e-3).no
        except ValueError:
            print("Error Improper Input! Use ASE Atoms/Crystal Object")
        lattice_sub =""
        if sg < 3:
                lattice = "Triclinic"
        elif sg >= 3 and sg < 16:
            if sym == "P":
                lattice     = "Monoclinic"
                lattice_sub = "Simple"
            if sym == "C":
                lattice     = "Monoclinic"
                lattice_sub = "FC"
        elif sg >= 16 and sg < 75:
            if sym == "P":
                lattice = "Orthorombic"
                lattice_sub = "FC"
            if sym == "C":
                lattice = "Orthorombic"
                lattice_sub ="AB"
            if sym == "F":
                lattice = "Orthorombic"
                lattice_sub ="FC"
            if sym == "I":
                lattice = "Orthorombic"
                lattice_sub ="BC"
            if sym == "A":
                lattice = "Orthorombic"
                lattice_sub ="AB"
        elif sg >= 75  and sg < 143:
            if sym == "I":
                lattice = "Tetragonal"
                lattice_sub ="BC"
            if sym == "P":
                lattice = "Tetragonal"
                lattice_sub = "Simple"
        elif sg >= 143 and sg < 168:
            if sym == "P":
                lattice = "Hexagonal"
            if sym == "R":
                lattice = "Rhombohedral"
        elif sg >= 168 and sg < 195:
            lattice = "Hexagonal"
        elif sg >= 195:
            if sym == "P":
                lattice = "Cubic"
                lattice_sub ="Simple"
            if sym == "F":
                lattice = "Cubic"
                lattice_sub ="FC"
            if sym == "I":
                lattice = "Cubic"
                lattice_sub ="BC"
        #if lattice_sub == "":
        #    crys_system = "{}".format(lattice)
        #else:
        #    crys_system = "{}_{}".format(lattice,lattice_sub)
        return(lattice,lattice_sub)
    
    def minimalunitcell(self):
        """
        When Writing .d12 input files the minimal set of lattice parameters must be used
        when utilizing spacegroups.
          alpha = >bc
          beta  = >ac
          gamma = >ab
        """

        
        lattice,lattice_sub = self.Check_Sym()
        
        cell = self.crystal.cell.cellpar()
        a = cell[0]
        b = cell[1]
        c = cell[2]
                
        alpha = cell[3]
        beta  = cell[4]
        gamma = cell[5]


        if   lattice == "Cubic":
            unit_cell = "%-8.6f"%(a)

        elif lattice == "Hexagonal":
            unit_cell = "%-8.6f\t%-8.6f"%(a,c)

        elif lattice == "Hexagonal":
            unit_cell = "%-8.6f\t%-8.6f"%(a,c)
        elif lattice == "Rhombohedral":
            unit_cell = "%-8.6f\t%-8.6f"%(a,alpha)

        elif lattice == "Tetragonal":
            unit_cell = "%-8.6f\t%-8.6f"%(a,c)

        elif lattice == "Orthorhombic":
            unit_cell = "%-8.6f\t%-8.6f\t%-8.6f"%(a,b,c)

        elif lattice == "Monoclinic":
            # Find Unique lattice constant from the first 3
            ## - Find List Location 
            ## - Determine if a,b,c
            ## - Set for below
            id = np.where(unique([a,b,c]))[0][0]
            #print("ID",id)
            if id==0: #a unique
                unit_cell = "%-8.6f\t%-8.6f\t%-8.6f\t%-8.6f"%(a,b,c,beta)
            elif id==1: #b unique
                unit_cell = "%-8.6f\t%-8.6f\t%-8.6\t%-8.6ff"%(a,b,c,gamma)
            elif id==2: #c unique
                unit_cell = "%-8.6f\t%-8.6f\t%-8.6f\t%-8.6f"%(a,b,c,alpha)
        elif lattice == "Triclinic":
            unit_cell = "%-8.6f\t%-8.6f\t%-8.6f\t%-8.6f\t%-8.6f\t%-8.6f"%(a,b,c,alpha,beta,gamma)
        else:
            unit_cell = "%-8.6f\t%-8.6f\t%-8.6f\t%-8.6f\t%-8.6f\t%-8.6f"%(a,b,c,alpha,beta,gamma)
        return(unit_cell) 
    
    def Cell(self):
        
        fract = self.crystal.get_scaled_positions()
        ans   = self.crystal.get_atomic_numbers()
        ECPS  = [26,34,35,37,38,39,40]
        self.ecps = []
        
        cell = self.crystal.cell.cellpar()
        a = cell[0]
        b = cell[1]
        c = cell[2]
                
        alpha = cell[3]
        beta  = cell[4]
        gamma = cell[5]
        
        for an in ans:
            if an in ECPS or an > 1:
                self.ecps.append(an+200)
            else:
                self.ecps.append(an)
        d12_Cell = ""
        #d12_Cell += str(self.sg)+"\n"
        if self.sg != 1:
            d12_Cell += self.minimalunitcell()
        else:
            if self.form == 'SLAB':
                d12_Cell += "%-8.6f\t%-8.6f\t%-8.6f"%(a,b,gamma)
            else:
                for i in range(0,len(cell)):
                     d12_Cell += "{0:0.8f}  ".format(cell[i])    
        
        d12_Cell += "\n"
        
        d12_Cell += str(len(fract))+"\n"
        for i in range(0,len(fract)):
            d12_Cell += "{}  ".format(self.ecps[i])
            for j in range(0,len(fract[i])):
                if j == 2 and self.form =='SLAB':
                    if fract[i,j] >0.5:
                        d12_Cell +=  "{0:0.8f}   ".format(fract[i,j]*500.-500.)
                    else:
                        d12_Cell +=  "{0:0.8f}   ".format(fract[i,j]*500.)
                else:
                    d12_Cell +=  "{0:0.8f}   ".format(fract[i,j])
            d12_Cell += "\n"
        return(d12_Cell)
    
    def Basis(self):
        #LOGIC BLOCK FOR ATOMIC NUMBERS BASED ON BASIS SET LIBRARY
        #
        ##########################################################
        #Directory Where Basis Sets are Stored
        lib_basis = "/Users/williamcomaskey/Desktop/Projects/crys22/library/basis/"
        basis     = self.param["BASIS"]
        #TRIM TO UNIQUE ATOMIC NUMBERS
        atomic_num = unique(self.ecps)
        bs ="END\n"
        for i in range(0,len(atomic_num)):
            num     = atomic_num[i]
            basis_i = "{0:s}{1:s}/{2:d}".format(lib_basis,basis,num)
            f       = open(basis_i)
            bs      += f.read()
            f.seek(0)
            if f.read().endswith('\n'): pass
            else: bs      +='\n' 
            f.close()
        bs +="99 0\nEND\n"
        return(bs)
    def make_SLABCUT(self):
        d12_SLABCUT = ""
        fract = self.crystal.get_scaled_positions()
        d12_SLABCUT+="SLABCUT"+"\n"
        d12_SLABCUT+="0 0 1"+"\n"
        d12_SLABCUT+="1 {}".format(len(unique(fract[:,2])))+"\n"
        return(d12_SLABCUT)
        
    def DFT(self):
        # ADD LINES PF TEXT FOR TWO COMP
        # - SOC: If 2C is turned on then we must change the dictionary to turn off symmetry, optimizations, Range separated Hybrids etc
        #        We must also create a SOPSEUD file      
        # - SPIN: If TWOCOMP or SOC are selcted SPIN is turned off
        # EXTRA VALUES WE NORMALLY USE SUCH AS PRECISION       
        if self.param == '0':
            #Use Default Parameters set here
            a,b,c,alpha,beta,gamma = self.crystal.cell.cellpar()              
            ks      = [2,3,6,9,10,12,15]
            ka = kb = kc =1
            for k in ks:
                if k*a > L and k*a<R and ka ==1: ka = k
                if k*b > L and k*b<R and kb ==1: kb = k
                if k*c > L and k*c<R and kc ==1: kc = k
            IS= max(ka,kb,kc)
            ISP= 2*IS

            d12_DFT = "DFT\nSPIN\nPBE0-D3\nXXLGRID\nEND\nTOLINTEG\n8 8 8 10 34\nTOLDEE\n7\nSHRINK\n%d %d \nSCFDIR\nBIPOSIZE\n110000000\nEXCHSIZE\n110000000\nMAXCYCLE\n800\nFMIXING\n%d\\nPPAN\nEND"%(IS,ISP,FM)
        
        else:
            d12_DFT = ""
            if self.param["TWOCOMP"] == 1:
                d12_DFT +="TWOCOMPON"+"\n"
                d12_DFT +="GUESSPNOSO"+"\n"

            if self.param["SOC"]     == 1:
                d12_DFT +="SOC"+"\n"

            if (self.param["SOC"] == 1) or (self.param["TWOCOMP"] == 1):
                d12_DFT +="ENDTWO"+"\n"

            d12_DFT +="DFT"+"\n"          
            if (self.param["SPIN"] == 1) and (self.param["SOC"] != 1) or (self.param["TWOCOMP"] != 1):
                d12_DFT += "SPIN"+"\n"
                       
            # Check which functional to be used i.e B3LYP,PBE0,HSE06, etc.
            # - HSE06 (Range Separated Hybrid)can not be used if the TWOCOMP code is selected
            if self.param["Functional"]:             
                if self.param["Dispersion"]:
                    d12_DFT += self.param["Functional"]+"-"+self.param["Dispersion"]+"\n"
                else:
                    d12_DFT += self.param["Functional"]+"\n"

            # Check Size of Grid default in Crystal is XLGRID              
            if self.param["Grid"]:
                d12_DFT += self.param["Grid"]+"\n"             
            d12_DFT +="END"+"\n"
            if self.param["K RES"]:        
                # Shrink Factor
                
                if self.param["K RES"] =='LOW':
                    ISP    = 30
                    L      = 40.
                    R      = 60.
                    ks     = [2,3,6,9,10,12,15]
                if self.param["K RES"] =='MED':
                    ISP    = 60
                    L      = 60.
                    R      = 100.
                    ks     = [2,3,6,9,10,12,15,18,20,24,30]
                if self.param["K RES"] =='HIGH':
                    ISP    = 100
                    L      = 100.
                    R      = 450.
                    ks     = [2,3,6,9,10,12,15,18,20,24,30,36,42]
            else: 
                ISP    = 30
                L      = 40.
                R      = 60.
                ks     = [2,3,6,9,10,12,15]

            a,b,c,alpha,beta,gamma = self.crystal.cell.cellpar()             
            IS = IS1 = IS2 = IS3 = 1
            if self.sg == 1:
                for k in ks:
                    if k*a > L and k*a<R and IS1 == 1: IS1 = k
                    if k*b > L and k*b<R and IS2 == 1: IS2 = k
                    if k*c > L and k*c<R and IS3 == 1: IS3 = k
                IS= max(IS1,IS2,IS3)
                ISP= 2*IS
                d12_DFT +="TOLINTEG\n7 7 7 9 30\nTOLDEE\n9\nSHRINK\n{0:d} {1:d}\n".format(IS,ISP)        
            
            elif self.sg != 1 or self.param["SPACEGROUP"]:
                x = min(a,b,c)
                for k in ks:
                    if k*x > L and k*x<R and IS == 1: IS = k
                ISP =2*IS
                d12_DFT +="TOLINTEG\n7 7 7 9 30\nTOLDEE\n9\nSHRINK\n{0:d} {1:d}\n".format(IS,ISP)        


            d12_DFT +="SCFDIR\nMADELIND\n100\nBIPOSIZE\n110000000\nEXCHSIZE\n110000000\nMAXCYCLE\n800\n"
            if self.param["Fock-Mixing"]:
                FM = self.param["Fock-Mixing"]
            else:
                FM = 50
            d12_DFT += "FMIXING\n{0:d}".format(FM)
            d12_DFT += "\nPPAN\nEND"

        return(d12_DFT)
                       
    def D12(self):
        #GLUE TOGETHER EVERYTHING 
        SLABCUT=True
        d12_Header   = self.Header()
        d12_Cell     = self.Cell()
        d12_Optimize = self.Optimize()
        d12_Basis    = self.Basis()
        d12_DFT      = self.DFT()            
        d12_string   = d12_Header+d12_Cell
        if SLABCUT == True:
            d12_string   += self.make_SLABCUT()  
        d12_string   +=d12_Optimize+d12_Basis+d12_DFT
        return(d12_string)
    
    def Write(d12_string, file):
        with open(file, 'w') as f:
            f.write(d12_string)
    def Crystal2Cif(self,file,fmt="cif"):
        a,b,c,alpha,beta,gamma = self.crystal.cell.cellpar()
        if c > 499.0:
            pass
        else:
            self.crystal.write(file,format=fmt)
    #def Sopseud(self):
        
    #What do I want the print output to show
    #def __repr__():
    #    pass
    #def __str()__:
    #    pass
    
    # Method for lazily adding layers as heterostructures
    #def __add__(self,layer):
    #    pass\
def write_Output2D12(file, in_type = "OPT",out_type="1C",form='CRYSTAL'):
    outfile = file.replace(".out",".d12")
    outfile = outfile.replace(in_type,out_type)
    output  = read_Output(file)
    crysout = Crystal22(output,form = form, param =out_type)
    
    #with open(outfile,'w') as f:
    #    f.write(crysout.D12())
    print(crysout.D12())


In [3]:
file = '/Users/williamcomaskey/Desktop/Projects/Bi_Final/Bi_111_1L_OPT_LDA_a.out'

#dir = r'/Users/williamcomaskey/Desktop/Crystal22/Bi_111/' 
#write_Output2D12(file,in_type = "OPT",out_type="1C",form='SLAB')


In [4]:
DIR = "/Users/williamcomaskey/Desktop/HSCIFS/"
files = glob.glob(dir+"*.cif")

for file in files:
    write_Output2D12(file)

TypeError: unsupported operand type(s) for +: 'builtin_function_or_method' and 'str'

In [ ]:
DIR = "/Users/williamcomaskey/Desktop/"




In [4]:
import glob
import ase
DESK = "/Users/williamcomaskey/Desktop/"
DIR ="/Users/williamcomaskey/Desktop/"
DIR = "/Users/williamcomaskey/Dropbox/Papers/2022_Collaboration_with_MagLab_LuisBalicas/Early_discussions_Files/"
file ="Rh3Si4_No75_Shirin.cif"
DIR ="/Users/williamcomaskey/Desktop/HSCIFS/YS/"

file ="CuBr-AuBr_SYM4_slab.cif"

#DIR = "/Users/williamcomaskey/Desktop/BALICAS/"
#files = glob.glob(DIR+"*.cif")
files = glob.glob(DIR+file)


for file in files:
    #READ CIF TO ASE
    struc = ase.io.read(file)
    print(get_spacegroup(struc).no)
    form = "CRYSTAL"
    out_type = "OPT"
    #ASE to CRYS22
    crysout = Crystal22(struc,form = form, param =out_type)
    #CRYS22 to D12
    #print(crysout.D12())
    file_d12 = file.replace(".cif","_OPT.d12")
    file_d12 = file_d12.replace(DIR,DESK)
    print(file)
    with open(file_d12,'w') as f:
        f.write(crysout.D12())


3
/Users/williamcomaskey/Desktop/HSCIFS/YS/CuBr-AuBr_SYM4_slab.cif


In [2]:
def get_SOPSEUD(ECP):
    DIR = "/Users/williamcomaskey/Desktop/Projects/crys22/library/basis/sopseud/"
    SOP = DIR+"{}.mol".format(ECP)
    with open(SOP) as f:
        data = f.read()
    N = int(data.splitlines()[0].split(';')[0].split(',')[-1])
    ele = data.splitlines()[0].split(';')[0].split(',')[1]
    if(N==0):
        #print("No Pseudo Potential for {0:s}".format(ele))
        #exit(1)
        return(False)
    sopseud = "STUTTGART {0}\n".format(N)
    Ls = ['S','P','D','F','G']
    for i in range(-N,-0):
        l = i+N+1

        M = int(data.splitlines()[i].split(";")[0])
        sopseud = sopseud+"{0} {1:n} {2:.0f}.\n".format(Ls[l],M,1.0)
        Data_i = data.splitlines()[i].split(";")
        #print(Data_i)
        for j in range(0,M):
            Data_j = Data_i[j+1].split(",")
            r = int(Data_j[0])
            A = float(Data_j[2])*(2.*l+1.)/2.
            B = float(Data_j[1])
            sopseud = sopseud+" {0:n}  {1:12.7f}   {2:10.7f}\n".format(r-2,A,B)
    return(sopseud)

In [1]:
from numpy import unique
def SOPSEUD(ECPS,fname="",dir=""):
    uECPS = unique(ECPS)
    with open(dir+fname,'w') as f:
        for i, ecpi in enumerate(uECPS):
            try:
                get_SOPSEUD(ecpi)
                for j, ecpj in enumerate(ECPS):
                    if ecpi == ecpj:
                        f.write("{} ".format(j+1))
                f.write('\n')
                f.write(get_SOPSEUD(ecpi))
            except:
                continue

ECPS  = [214,214,214,214,227,227,227,227]
#fname = "test.sopseud"

DIR = "/Users/williamcomaskey/Desktop/2DGENOME/"
fname = "CoSi_SOC.sopseud"
SOPSEUD(ECPS,fname = fname,dir=DIR)

#files = glob.glob(DIR+"*SOC.d12")
#SOPSEUD(ECPS,fname = fname,dir="")

# for file in files:
#     ECPS = []
#     N    = 100
#     with open(file) as f:
#         print(file)
#         for i, line in enumerate(f.readlines()):
#             if i==4:
#                 N = int(line)
#             if i>4 and i<5+N:
#                 ECPS.append(int(line.split()[0]))
#     fname = file.replace(".d12",".sopseud")            
#     SOPSEUD(ECPS,fname = fname,dir="")



### PIPELINE TEST (w/o SBATCH)

In [20]:
from ase.build import bulk
import numpy as np
import os
from ase import Atoms
from ase.calculators.emt import EMT
from ase.constraints import FixAtoms
from ase.optimize import QuasiNewton
from ase.build import fcc111, add_adsorbate
from numpy import sqrt
from ase.spacegroup.symmetrize import *
from ase.spacegroup.xtal import crystal

a     = 4.19
c     = 11.97547308
print(a)
alpha = 90
gamma =120
Bi_bulk = crystal('Bi2', 
                [(0.00000000,  0.00000000,  0.26721600)],
                spacegroup =166,
                cell=[a,a,c,alpha,alpha,gamma])
ECPS  = [221,221,223,240,240]
fname = "test.sopseud"
submit_name ="test"
DIR = "/Users/williamcomaskey/Desktop/Projects/INPUT/"
#------------------------------------------------------------------------------------------------------
#----------------------------------  STEP 1: OPT    ---------------------------------------------------
#------------------------------------------------------------------------------------------------------

#CREATE UNIFORM NAMING SCHEME
material = "Bi_bulk"

#CREATE OPT.d12
submit_OPT = "{}_OPT".format(material)
d12_OPT    = "{}.d12".format(submit_OPT)

New    = Crystal22(Bi_bulk,form = 'CRYSTAL', param ="OPT")
with open(DIR+d12_OPT,'w') as f:  
    f.write(New.D12())

#EXECUTE OPT.d12
# We can just take what we need from this .py for the singular case
os.system("/Users/williamcomaskey/Desktop/Projects/crys22/scripts/submitcrystal22.sh " + submit_OPT + " 24")
#exec(open(submit).read())

#------------------------------------------------------------------------------------------------------
#---------------------------------  STEP 2: 1C    -----------------------------------------------------
#------------------------------------------------------------------------------------------------------
submit_1C = "{}_1C".format(material)
d12_1C    = "{}.d12".format(submit_1C)
out_OPT   = "{}.out".format(submit_OPT)

#READ OPT.out --DONE--
#CREATE 1C.d12 --DONE-- write_Output2D12
write_Output2D12(out_OPT, in_type = "OPT",out_type="1C",form='CRYSTAL')

#Execute 1C.d12 --DONE-- submitcrystal22.sh
os.system("/Users/williamcomaskey/Desktop/Projects/crys22/scripts/submitcrystal22.sh " + submit_1C + " 24")
#exec(open(submit).read())

#------------------------------------------------------------------------------------------------------
#--------------------------------   STEP 3: SOC    ----------------------------------------------------
#------------------------------------------------------------------------------------------------------
submit_SOC = "{}_SOC".format(material)
d12_SOC    = "{}.d12".format(submit_SOC)

#Create SOC.d12 --DONE-- write_Output2D12
write_Output2D12(out_OPT, in_type = "OPT",out_type="SOC",form='CRYSTAL')

#Copy 1C.f9 and 1C.f20 as SOC.f9 and SOC.f20  --DONE-- for pipeline case we dont need loop
f91C   = "{}.f9".format(submit_1C)
f9SOC  = "{}.f9".format(submit_SOC)
f20SOC = "{}.f20".format(submit_SOC)

#cd to working directory
os.system("cd crystal")
os.system("cp {} INPUT/{}".format(f91C,f9SOC))
os.system("cp INPUT/{} INPUT/{}".format(f9SOC,f20SOC))

#Create SOC.sopseud --DONE--
SOP = f9SOC.replace('f9','sopseud')
SOPSEUD(ECPS,fname=SOP,dir=DIR)

#Execute SOC.d12 --DONE-- submitcrystal22.sh
os.system("/Users/williamcomaskey/Desktop/Projects/crys22/scripts/submitcrystal22.sh " + submit_SOC + " 24")
#exec(open(submit).read())

#------------------------------------------------------------------------------------------------------
#------------------------------   STEP 4: BAND/DOSS    ------------------------------------------------
#------------------------------------------------------------------------------------------------------
#Functionalize Kevin code for a single use case
#Add Energy Range for DOSS
#READ SHRINK onto DOSS



4.19


In [21]:
print(os.getcwd())

/Users/williamcomaskey/Desktop/Projects/crys22/examples


In [30]:
#readfeatures(DIR)

In [8]:
from pathlib import Path
import warnings
import numpy as np
import glob
import math
import matplotlib
matplotlib.use('tkagg') # <-- THIS MAKES IT FAST!
import matplotlib.pyplot as plt
import csv
import os
import math

def nCr(n,r):
    f = math.factorial
    return f(n) / f(r) / f(n-r)

DIR      = "/Users/williamcomaskey/Desktop/HSE06-A/"
pathlist = sorted(glob.glob(DIR+'*_SLAB.out'))
ntype    = len("_BULK.out")

# DIR      = "/Users/williamcomaskey/Desktop/HSE06-B/"
# pathlist = sorted(glob.glob(DIR+'*_slab.out'))
# ntype    = len("_bulk.out")

nDIR     = len(DIR)
DATA     = [[],[],[]]
#FIN    = " FINAL OPTIMIZED GEOMETRY"                   # switch on
LAT    = " LATTICE PARAMETERS  (ANGSTROMS AND DEGREES) - PRIMITIVE CELL"
CART   = " CARTESIAN COORDINATES - PRIMITIVE CELL"
ENERGY = " * OPT END - CONVERGED * E(AU):"
INDGAP = " INDIRECT ENERGY BAND GAP:"
DIRGAP = " DIRECT ENERGY BAND GAP:"
HeV    = 27.2114
#################################################################################
# # of sym ops len(sg.get_symop())
# unique_sites(self,scaled_positions,symprec=1e-3,output_mask=False,map_to_unitcell=True)
with open(DIR+"PROPERTIES.csv","w") as out:
  writer = csv.writer(out)
  #               table = [material,Ebg_slb,GAPTYPE_slb,Ebg_blk,GAPTYPE_blk,E_slb,E_blk,Eb,IAD_slb,IAD_blk,state_slb,state_blk,dens,mass,atoms,space]
  writer.writerow(["Material","SG(slab)","Eg (slab)","Type","SG(bulk)","Eg (bulk)","Type","E (slab)","E (bulk)","Eb","IAD (slab)","IAD (bulk)","State (slab)","State (bulk)","Dens (bulk)","Mass","Atoms","Vacuum"])
  for path in pathlist:
      # because path is object not string
      path_in_str = str(path)
      material = path_in_str[nDIR:-ntype]
      if material == "":break
      #print(material)

      FERMI    = "   FERMI ENERGY:"
      DONE     = " * OPT END - CONVERGED"
      GEOM     = " FINAL OPTIMIZED GEOMETRY"
      COND     = " POSSIBLY CONDUCTING STATE"

#       SLAB     = material+"_slab.out"
#       BULK     = material+"_bulk.out"
      
      SLAB     = material+"_SLAB.out"
      BULK     = material+"_BULK.out"
    


      if os.path.exists(DIR+SLAB) and os.path.exists(DIR+BULK):

        
        #print(len(sg_bulk.get_symop()))
        
        print(material)
        try:
            with open(DIR+SLAB,'r') as slb, open(DIR+BULK,'r') as blk:
              #:::::::::::::::   SLAB ANALYSIS   ::::::::::::::::::::::
              el = np.zeros(99)
              x  = np.zeros(99)
              y  = np.zeros(99)
              z  = np.zeros(99)

              id  = 9999
              md  = 9999
              atoms    = 99
              k        = 0
              switch   = 0
              E_slb    = 999999
              IAD_slb  = 999999
              state_slb    = "Null"
              Ebg_slb      = 999999
              GAPTYPE_slb  = "Null"
              for i, line in enumerate(slb):
                # GET CARTESIAN POSTIONS FOR ATOMS
                if line.startswith(" NUMBER OF IRREDUCIBLE"):
                  atoms = int(float(line.split()[-1]))
                if line.startswith(CART):
                  md    = i 
                if " PROCESS" in line: 
                  md+=1
                  continue
                if md+4 <= i and i <= md+atoms+3:
                   el[i-md-5] = float(line.split()[-5])
                   if el[i-md-5] >200:el[i-md-11] -=200
                   x[i-md-5] = float(line.split()[-3])
                   y[i-md-5] = float(line.split()[-2])
                   z[i-md-5] = float(line.split()[-1])

                # GET THE INFORMATION ON THE PRIMITIVE CELL (ANG. and DEG.)
                if LAT in line:
                  id = i  
                if i == id+2:
                    # FIND THE OPTIMIZED PRIMITIVE CELL
                    cell    = line.split()
                    A       = float(cell[0])
                    B       = float(cell[1])
                    C       = float(cell[2])
                    alpha   = float(cell[3])
                    beta    = float(cell[4])
                    gamma   = float(cell[5])
                    V       = float(cell[6])

                if line.startswith(COND):
                  state_slb = "COND"
                  Ebg_slb   = 0.


                # TOTAL ENERGY OF THE SLAB
                if "ETOT(AU)" in line:
                  E_slb = float(line.split()[3])
                  #print("ETOT(AU)",E_slb)


                # BANDGAP FOR SLAB
                if line.startswith(INDGAP) or line.startswith(DIRGAP):
                  Ebg_slb = float(line.split()[-2])
                  if line.startswith(INDGAP):
                    GAPTYPE_slb = "INDIRECT"
                  if line.startswith(DIRGAP):
                    GAPTYPE_slb = "DIRECT"
                  #print(line)

                if Ebg_slb <9. and Ebg_slb >0.:     state_slb = "SEMI"
                if Ebg_slb >9. and Ebg_slb < 10000: state_slb = "INSU"
                if Ebg_slb >100: continue
                N   = float(nCr(atoms,2))
                d        = 0.
                for i in range(0,atoms):
                   for j in range(i,atoms):
                      d += np.sqrt((x[i]-x[j])**2.+(y[i]-y[j])**2.+(z[i]-z[j])**2)
                IAD_slb  = d/N
                mass     = abs(sum(el))
                #el = np.zeros(40)


              #:::::::::::::::   BULK ANALYSIS   ::::::::::::::::::::::
              el = np.zeros(99)
              x  = np.zeros(99)
              y  = np.zeros(99)
              z  = np.zeros(99)

              id  = 9999
              md  = 9999         
              atoms    = 99
              d        = 0.
              k        = 0
              switch   = 0
              E_blk    = 999999
              IAD_blk  = 999999
              state_blk    = "Null"
              Ebg_blk      = 9999999
              GAPTYPE_blk  = "Null"
              for i, line in enumerate(blk):
                # GET CARTESIAN POSTIONS FOR ATOMS
                if line.startswith(" NUMBER OF IRREDUCIBLE"):
                  atoms = int(float(line.split()[-1]))
                if line.startswith(CART):
                  md    = i 
                if " PROCESS" in line: 
                  md+=1
                  continue
                if md+4 <= i and i <= md+atoms+3:
                   el[i-md-5] = float(line.split()[-5])
                   if el[i-md-5] >200:el[i-md-11] -=200
                   x[i-md-5] = float(line.split()[-3])
                   y[i-md-5] = float(line.split()[-2])
                   z[i-md-5] = float(line.split()[-1])

                # GET THE INFORMATION ON THE PRIMITIVE CELL (ANG. and DEG.)
                if LAT in line:
                  id = i  
                if i == id+2:
                    # FIND THE OPTIMIZED PRIMITIVE CELL
                    cell    = line.split()
                    A       = float(cell[0])
                    B       = float(cell[1])
                    C       = float(cell[2])
                    alpha   = float(cell[3])
                    beta    = float(cell[4])
                    gamma   = float(cell[5])
                    V       = float(cell[6])

                if line.startswith(COND):
                  state_blk = "COND"
                  Ebg_blk   = 0.


                # TOTAL ENERGY OF THE BULK
                if "ETOT(AU)" in line:
                  E_blk = float(line.split()[3])
                  #print("ETOT(AU)",E_blk)


                # BANDGAP FOR BULK
                if line.startswith(INDGAP) or line.startswith(DIRGAP):
                  Ebg_blk = float(line.split()[-2])
                  if line.startswith(INDGAP):
                    GAPTYPE_blk = "INDIRECT"
                  if line.startswith(DIRGAP):
                    GAPTYPE_blk = "DIRECT"
                  #print(line)

              if Ebg_blk <9. and Ebg_blk >0.:     state_blk = "SEMI"
              if Ebg_blk >9. and Ebg_blk < 10000: state_blk = "INSU"
              if Ebg_blk >100: continue
              N   = float(nCr(atoms,2))
              d   = 0.
              for i in range(0,atoms):
                 for j in range(i,atoms):
                    d += np.sqrt((x[i]-x[j])**2.+(y[i]-y[j])**2.+(z[i]-z[j])**2)
              IAD_blk  = d/N
              mass     = sum(el)

              space  =  (C+min(z)) - max(z)
              Eb     = (E_blk-E_slb)*HeV
              dens   = abs(float(mass)/V)
              table = [material,sg_slab,Ebg_slb,GAPTYPE_slb,sg_bulk,Ebg_blk,GAPTYPE_blk,E_slb,E_blk,Eb,IAD_slb,IAD_blk,state_slb,state_blk,dens,mass,atoms,space]
              writer.writerow(table)
            #GET SPACEGROUPS    
            output_slab   = read_Output(DIR+SLAB)
            sg_slab       = get_spacegroup(output_slab, symprec=1e-01).no
        
            #print(len(sg_slab.get_symop()))

            output_bulk   = read_Output(DIR+BULK)
            sg_bulk       = get_spacegroup(output_bulk, symprec=1e-01).no
        except Exception as e: 
            print(e)
            continue
out.close()

Ag1P15


KeyboardInterrupt: 